In [ ]:
import bluetooth
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 0
from testsHelper import initBluetoothSocket, getSocketData, AnimationWorker

# Bluetooth Test

In [ ]:
import numpy as np
dataStr = "@\n@1,3,4,5,6,2@"
data = dataStr.splitlines()
data = filter(lambda row: row[0] == "@" and row[-1] == "@", data)
data = map(lambda x: list(x.strip("@").split(",")), data)
data = np.array(list(data), dtype=np.float64)
data

In [ ]:
devices = bluetooth.discover_devices(lookup_names=True, duration=2)
devices

In [ ]:
sock = initBluetoothSocket(address="8C:CE:4E:8A:91:0A")

In [ ]:
time, roll, pitch, yaw = getSocketData(sock)

In [ ]:
sp1 = 313
sp2 = 312
sp3 = 311

fig = plt.figure(figsize=(12,7))
# initialize subplot1
ax1 = fig.add_subplot(sp1)
ax1.plot(time, roll)
ax2 = fig.add_subplot(sp2)
ax2.plot(time, pitch)
ax3 = fig.add_subplot(sp3)
ax3.plot(time, yaw)

In [ ]:
sock.send("Hello")

# Motion Test

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
 
def plot_basic_object(points):
    """Plots a basic object, assuming its convex and not too complex"""
    tri = Delaunay(points).convex_hull
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    S = ax.plot_trisurf(points[:,0], points[:,1], points[:,2],
                        triangles=tri,
                        shade=True, cmap=cm.Blues,lw=0.5)
    ax.set_xlim3d(-5, 5)
    ax.set_ylim3d(-5, 5)
    ax.set_zlim3d(-5, 5)
     
def create_cube(side_length=5):
    return create_cuboid(side_length, side_length, side_length)  

def create_cuboid(dx, dy, dz):
    """Creates a cube starting from the given bottom-lower point (lowest x, y, z values)"""
    bottom_lower = np.array([-dx/2, -dy/2, -dz/2])
    points = np.vstack([
        bottom_lower,
        bottom_lower + [0, dy, 0],
        bottom_lower + [dx, dy, 0],
        bottom_lower + [dx, 0, 0],
        bottom_lower + [0, 0, dz],
        bottom_lower + [0, dy, dz],
        bottom_lower + [dx, dy, dz],
        bottom_lower + [dx, 0, dz],
        bottom_lower,
    ])
    return points
 
 
cube_1 = create_cube(side_length=2)
amount = np.array([3, -3, 0])
plot_basic_object(cube_1)

translated_cube = amount + cube_1
  
plot_basic_object(translated_cube)
plt.show()

In [ ]:
from __future__ import division
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import numpy as np

from matplotlib import patches
from scipy.spatial.transform import Rotation as R

left_cube = cube_1 + np.array([-0.1, 0, 0])
right_cube = cube_1 + np.array([0.1, 0, 0])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(left=-0.2, right=0.2)
ax.set_ylim3d(-0.1, 0.1)
ax.set_zlim3d(-0.05, 0.2)

left_cube_tri = Delaunay(left_cube).convex_hull
plot = ax.plot_trisurf(left_cube[:,0], left_cube[:,1], left_cube[:,2],
                       triangles=left_cube_tri, shade=True, cmap=cm.Blues, lw=0.5)
right_cube_tri = Delaunay(right_cube).convex_hull
plot = ax.plot_trisurf(right_cube[:,0], right_cube[:,1], right_cube[:,2],
                       triangles=right_cube_tri, shade=True, cmap=cm.Blues, lw=0.5)

thetaLin = np.deg2rad(np.linspace(0, 60, 100))

cuboid = create_cuboid(0.05, 0.03, 0.03)

def data_gen(framenumber, thetaLin, plot):
    ax.clear()
    ax.set_xlim3d(left=-0.2, right=0.2)
    ax.set_ylim3d(-0.05, 0.05)
    ax.set_zlim3d(-0.05, 0.2)
    ax.set_aspect('equal')

    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.zaxis.set_ticklabels([])

    for line in ax.xaxis.get_ticklines():
        line.set_visible(False)
    for line in ax.yaxis.get_ticklines():
        line.set_visible(False)
    for line in ax.zaxis.get_ticklines():
        line.set_visible(False)

    theta = thetaLin[framenumber]

    left_cube = cuboid + np.array([-0.1*np.cos(theta), 0.1*np.sin(theta), 0])
    # right_cube = cuboid + np.array([0.1*np.cos(theta), 0.1*np.sin(theta), 0])
    right_cube = cuboid + np.array([0.15, 0, 0])

    left_r = R.from_rotvec([0, theta, 0])
    # right_r = R.from_rotvec([0, -theta, 0])
    left_cube = left_r.apply(left_cube)
    # right_cube = right_r.apply(right_cube)

    left_cube_tri = Delaunay(left_cube).convex_hull
    plot = ax.plot_trisurf(left_cube[:,0], left_cube[:,1], left_cube[:,2],
                    triangles=left_cube_tri, shade=True, cmap=cm.Blues, lw=0.5)
    right_cube_tri = Delaunay(right_cube).convex_hull
    plot = ax.plot_trisurf(right_cube[:,0], right_cube[:,1], right_cube[:,2],
                    triangles=right_cube_tri, shade=True, cmap=cm.Blues, lw=0.5)
    return plot,


anim = animation.FuncAnimation(fig, data_gen, fargs=(thetaLin, plot),
                              interval=30, blit=False)
plt.draw()
plt.show()
anim.save("motion.mp4", dpi=200, writer="ffmpeg", fps=100)

# Integration Test

In [ ]:
np.concatenate([[[1,3,4],[1,2,4]], [[1,2,3,4],[1,2,1,2]]], axis=1)

In [ ]:
ani_worker = AnimationWorker(address="8C:CE:4E:8A:91:0A")
ani_worker.start()